## MATH 440 DNN for Discrete Time Optimal Stopping Problems
### Mar 29, 2025

In this section, we utilize DNN for Optimal Stopping rather than simple regression analysis.

In [18]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import LeakyReLU

### Paths Generation
Similar to LSM, we use the same codes for paths generation, i.e., follows the geometric Brownian Motions

In [19]:
def paths_mc_gbm(S_0,r,sigma,T,N,M):
    dt=T/N
    paths=np.zeros((M,N+1))#with each row denotes different paths and cols denote time series
    paths[:,0]=S_0
    for t in range(1,N+1):
        Z=np.random.normal(0, 1, M)
        paths[:,t]=paths[:,t-1]*np.exp((r-0.5*sigma**2)*dt+sigma*np.sqrt(dt)*Z)
    return paths

### Construct NN Models

In [20]:
def nn_model(input_dim,hidden_units,hidden_layers):
    model=Sequential()
    model.add(Input(shape=(input_dim,)))
    for _ in range(hidden_layers):
        model.add(Dense(hidden_units))
        model.add(LeakyReLU(alpha=0.3))
    model.add(Dense(1, activation='linear'))
    model.compile(optimizer=Adam(),loss='mse')
    return model

In [21]:
def nn_fit_model(S_0,K,r,sigma,T,N,M,hidden_units,hidden_layers,epochs,batch_size):
    dt=T/N
    paths=paths_mc_gbm(S_0,r,sigma,T,N,M)
    V=np.zeros_like(paths)
    V[:,-1]=np.maximum(K-paths[:,-1],0)
    exercise=np.zeros_like(paths,dtype=bool)
    exercise[:,-1]=V[:,-1] > 0

    scaler=StandardScaler()
    for t in range(N-1,0,-1):
        in_the_money=paths[:,t] < K
        X=paths[in_the_money,t].reshape(-1,1)
        Y=V[in_the_money,t+1]*np.exp(-r*dt)        
        if t == N-1:scaler.fit(X)      
        X_scaled=scaler.transform(X)

        model=nn_model(input_dim=1,hidden_units=hidden_units,hidden_layers=hidden_layers)
        model.fit(X_scaled,Y,epochs=epochs,batch_size=batch_size,verbose=0)        
        continuation=model.predict(X_scaled).flatten()
        exercise_value=K-X.flatten()
        exercise[in_the_money,t]=exercise_value>continuation
        V[in_the_money,t]=np.where(exercise[in_the_money,t],exercise_value,Y)
        V[~in_the_money,t]=V[~in_the_money,t+1]*np.exp(-r*dt)
    first_exercise=np.argmax(exercise, axis=1)
    payoff_times=np.where(exercise.any(axis=1),first_exercise,N)
    payoffs=V[np.arange(M), payoff_times]  
    return np.mean(payoffs*np.exp(-r * payoff_times * dt))

In [22]:
S_0=100
K=100
r=0.05
sigma=0.2
T=1 #time in total
N=50 #number of steps
M=10000 #number of paths

option_value=nn_fit_model(S_0,K,r,sigma,T,N,M,hidden_units=32,hidden_layers=2,epochs=5,batch_size=32)
print(f"Neural Network Option Value: {option_value:.4f}")

/Users/dxr/anaconda3/lib/python3.10/site-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


139/139 ━━━━━━━━━━━━━━━━━━━━ 0s 565us/step


/Users/dxr/anaconda3/lib/python3.10/site-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


140/140 ━━━━━━━━━━━━━━━━━━━━ 0s 553us/step


/Users/dxr/anaconda3/lib/python3.10/site-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


...

154/154 ━━━━━━━━━━━━━━━━━━━━ 0s 483us/step


/Users/dxr/anaconda3/lib/python3.10/site-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


155/155 ━━━━━━━━━━━━━━━━━━━━ 0s 486us/step


/Users/dxr/anaconda3/lib/python3.10/site-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


155/155 ━━━━━━━━━━━━━━━━━━━━ 0s 483us/step
Neural Network Option Value: 5.8970


Here, we implement a Simple Feedforward Neural Network with 2 hidden layers. To be consistent with the reference paper, we utilized LeakyReLU as the activation functions, but we didn't add regularization term here.